<a href="https://colab.research.google.com/github/macroact-lab/robotics-book/blob/main/3_2_maicat_rpy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title 1. 라이브러리 및 로봇 모델 정의 (실행 필수)
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation
from mpl_toolkits.mplot3d import Axes3D
from collections import OrderedDict
from IPython.display import HTML

# ==============================================================================
# 수학 및 기하학 라이브러리
# ==============================================================================
def RpToTrans(R, p):
    return np.r_[np.c_[R, p], [[0, 0, 0, 1]]]

def TransToRp(T):
    T = np.array(T)
    return T[0:3, 0:3], T[0:3, 3]

def RPY(roll, pitch, yaw):
    Rx = np.array([[1, 0, 0], [0, np.cos(roll), -np.sin(roll)], [0, np.sin(roll), np.cos(roll)]])
    Ry = np.array([[np.cos(pitch), 0, np.sin(pitch)], [0, 1, 0], [-np.sin(pitch), 0, np.cos(pitch)]])
    Rz = np.array([[np.cos(yaw), -np.sin(yaw), 0], [np.sin(yaw), np.cos(yaw), 0], [0, 0, 1]])
    return np.matmul(np.matmul(Rx, Ry), Rz)

def solve_2link_intersection(p_start, p_end, r1, r2, bend_direction):
    vec = p_end - p_start
    d = np.linalg.norm(vec)

    max_reach = r1 + r2
    if d >= max_reach * 0.999: d = max_reach * 0.999
    min_reach = abs(r1 - r2)
    if d <= min_reach * 1.001: d = min_reach * 1.001

    a = (r1**2 - r2**2 + d**2) / (2*d)
    h = np.sqrt(max(0, r1**2 - a**2))

    x2 = p_start + a * (vec / d)
    perp_vec = np.array([-vec[2], 0, vec[0]])
    if np.linalg.norm(perp_vec) == 0: perp_vec = np.array([0, 0, 1])
    perp_vec = perp_vec / np.linalg.norm(perp_vec)

    if bend_direction == 'backward': perp_vec = -perp_vec
    return x2 + h * perp_vec

# ==============================================================================
# Maicat 모델 클래스
# ==============================================================================
class MaicatModel:
    def __init__(self):
        self._len_pan = 0.016
        self._len_shoulder_f = 0.046
        self._len_leg_f = 0.063
        self._len_shoulder_r = 0.046
        self._len_bridge_r = 0.05
        self._len_leg_r = 0.025

        self._hip_x = 0.11
        self._hip_y = 0.084
        self._foot_x = 0.13
        self._foot_y = 0.11

        self._body_to_hip = OrderedDict()
        self._body_to_hip["FL"] = np.array([self._hip_x / 2.0, self._hip_y / 2.0, 0])
        self._body_to_hip["FR"] = np.array([self._hip_x / 2.0, -self._hip_y / 2.0, 0])
        self._body_to_hip["RL"] = np.array([-self._hip_x / 2.0, self._hip_y / 2.0, 0])
        self._body_to_hip["RR"] = np.array([-self._hip_x / 2.0, -self._hip_y / 2.0, 0])

        self._world_foot = OrderedDict()
        self._world_foot["FL"] = np.array([self._foot_x / 2.0, self._foot_y / 2.0, 0.0])
        self._world_foot["FR"] = np.array([self._foot_x / 2.0, -self._foot_y / 2.0, 0.0])
        self._world_foot["RL"] = np.array([-self._foot_x / 2.0, self._foot_y / 2.0, 0.0])
        self._world_foot["RR"] = np.array([-self._foot_x / 2.0, -self._foot_y / 2.0, 0.0])

def get_leg_points_geometric(model, key, T_wb):
    points = []
    p_bh = model._body_to_hip[key]
    T_bh = RpToTrans(np.eye(3), p_bh)
    T_wh = np.dot(T_wb, T_bh)
    _, p_hip = TransToRp(T_wh)
    p_foot = model._world_foot[key]

    side_sign = 1 if 'L' in key else -1
    R_wb, _ = TransToRp(T_wb)
    y_axis_body = R_wb[:, 1]
    p_pan = p_hip + y_axis_body * (model._len_pan * side_sign)

    points.append(p_hip)
    points.append(p_pan)

    if 'F' in key:
        p_knee = solve_2link_intersection(p_pan, p_foot, model._len_shoulder_f, model._len_leg_f, 'backward')
        points.append(p_knee)
        points.append(p_foot)
    else:
        l_long = model._len_shoulder_r + model._len_leg_r
        l_bridge = model._len_bridge_r
        p_temp_joint = solve_2link_intersection(p_pan, p_foot, l_long, l_bridge, 'forward')
        vec_thigh = p_temp_joint - p_pan
        u_t = vec_thigh / np.linalg.norm(vec_thigh)
        p_knee = p_pan + u_t * model._len_shoulder_r
        vec_bridge = p_foot - p_temp_joint
        p_ankle = p_knee + vec_bridge

        points.append(p_knee)
        points.append(p_ankle)
        points.append(p_foot)

    return np.array(points)

# ==============================================================================
# Colab용 애니메이션 생성 함수
# ==============================================================================
def create_maicat_animation(mode='roll', frames=50):
    model = MaicatModel()

    # 1. Figure 설정
    fig = plt.figure(figsize=(10, 6))
    ax = fig.add_subplot(111, projection='3d')

    # 바닥 그리드
    xx, yy = np.meshgrid(np.linspace(-0.3, 0.3, 10), np.linspace(-0.2, 0.2, 10))
    ax.plot_wireframe(xx, yy, np.zeros_like(xx), color='gray', alpha=0.1)

    # 축 설정
    ax.set_xlim(-0.2, 0.2)
    ax.set_ylim(-0.2, 0.2)
    ax.set_zlim(0, 0.3)
    ax.set_xlabel('X (Front)')
    ax.set_ylabel('Y (Left)')
    ax.set_zlabel('Z (Up)')
    ax.view_init(elev=20, azim=-70)
    try: ax.set_box_aspect([1, 1, 1])
    except: pass

    # 2. 그래픽 객체 초기화
    body_line, = ax.plot([], [], [], 'k-', linewidth=3, label='Body')
    head_point, = ax.plot([], [], [], 'go', markersize=10, label='Head')
    leg_lines = {key: ax.plot([], [], [], color='blue', linewidth=4, marker='o')[0] for key in ["FL", "FR", "RR", "RL"]}

    title = ax.set_title(f"Mode: {mode.upper()}")

    # 3. 업데이트 함수
    def update(frame):
        # 시간 t
        t = frame * 0.1
        angle = np.radians(15 * np.sin(t)) # -15도 ~ +15도

        roll, pitch, yaw = 0, 0, 0
        if mode == 'roll': roll = angle
        elif mode == 'pitch': pitch = angle
        elif mode == 'yaw': yaw = angle

        # 몸통 높이 (약간의 상하 움직임)
        z_height = 0.085 + 0.002 * np.sin(t * 2)

        # 위치 계산
        orn = [roll, pitch, yaw]
        pos = [0, 0, z_height]

        R_wb = RPY(orn[0], orn[1], orn[2])
        T_wb = RpToTrans(R_wb, pos)

        # 몸통 업데이트
        hip_world_pos = [np.dot(T_wb, np.append(model._body_to_hip[k], 1))[:3] for k in ["FL", "FR", "RR", "RL"]]
        hip_pos_arr = np.array(hip_world_pos)

        bx = np.append(hip_pos_arr[:, 0], hip_pos_arr[0, 0])
        by = np.append(hip_pos_arr[:, 1], hip_pos_arr[0, 1])
        bz = np.append(hip_pos_arr[:, 2], hip_pos_arr[0, 2])

        body_line.set_data(bx, by)
        body_line.set_3d_properties(bz)

        # 머리 업데이트
        front_center = (hip_pos_arr[0] + hip_pos_arr[1]) / 2
        head_point.set_data([front_center[0]], [front_center[1]])
        head_point.set_3d_properties([front_center[2]])

        # 다리 업데이트
        for key in ["FL", "FR", "RR", "RL"]:
            points = get_leg_points_geometric(model, key, T_wb)
            leg_lines[key].set_data(points[:, 0], points[:, 1])
            leg_lines[key].set_3d_properties(points[:, 2])

        return body_line, head_point

    # 애니메이션 생성
    anim = animation.FuncAnimation(fig, update, frames=frames, interval=50, blit=False)
    plt.close() # 정적 그래프 출력 방지
    return HTML(anim.to_jshtml())

In [ ]:
# @title 2. Roll (좌우 갸웃) 움직임 확인
display(create_maicat_animation(mode='roll', frames=60))

In [ ]:
# @title 3. Pitch (앞뒤 끄덕) 움직임 확인
display(create_maicat_animation(mode='pitch', frames=60))

In [ ]:
# @title 4. Yaw (도리도리) 움직임 확인
display(create_maicat_animation(mode='yaw', frames=60))